In [ ]:
import snowflake.connector

In [ ]:
# Please check your creds before running the api!
'''snowflake_creds = {
    'user': 'YOUR_USERNMAE',
    'password': 'YOUR_PASS',
    'account': 'YOUR_ACC',
}

warehouse = {
    'warehouse': 'YOUR_WAREHOUSE',
}

role = {
    'role': 'YOUR_ROLE',
}'''

In [ ]:
import pandas as pd
import snowflake.connector
import json
#imported creds from different .ipynb for secuirity reasons
%run config.ipynb
from flask import Flask, request


app=Flask(__name__)

conn = snowflake.connector.connect(**snowflake_creds)
    
statement_1='use warehouse '+ warehouse['warehouse']
statement4="use role "+ role['role']

def run_query(conn, query):
    print("Executing the {} query".format(query))
    cursor = conn.cursor()
    try:
        cursor.execute(query)
    except Exception as e:
        print(e)
    cursor.close()


def run_query1(conn, query):
    cursor = conn.cursor()
    cursor.execute(query)
    records=str(cursor.fetchone()[0])
    cursor.close()
    return records


@app.route('/')
def hello_world():
    return """
    <h1>Welcome to the Snowflake API!</h1>
    <p>This API provides two main functions: Aggregator and Data Fetcher.</p>
    
    <h2>Aggregator Function:</h2>
    <p>Access: /aggregator</p>
    <p>Use: Specify your SQL query as 'query' parameter.</p>
    <p>Example: /aggregator?query=select count(*) from MORE_DATA.PROD.COVID_SYMPTHOMS;</p>
    
    <h2>Data Fetcher Function:</h2>
    <p>Access: /data_fetcher</p>
    <p>Use: Specify your SQL query as 'query' parameter.</p>
    <p>Example: /data_fetcher?query=select * from more_data.prod.covid_sympthoms where contact_yes = 1 and age_60 = 1;</p>

    <h2>Important!</h2>
    <p>Aggrerator returns only one value! So no gruop by etc. functions cannot be used! For that use Data Fetcher</p>
    <p>Data Fetcher returns json format</p>
    <p>Don't forget you specify your DATABASE.SCEHMA.TABLE otherwise it won't work</p>
    <p>Tested on MICROSOFT EDGE, SAFARI doesn't support this</p>

    <p>More examples:</p>
    <p>1. /data_fetcher?query=select * from more_data.prod.covid_sympthoms;</p>
    <p>2. /data_fetcher?query=select COUNTRY_REGION, SUM(CASES) AS Cases FROM COVID19_EPIDEMIOLOGICAL_DATA.PUBLIC.ECDC_GLOBAL GROUP BY COUNTRY_REGION;</p>
    <p>3. /data_fetcher?query=select * from more_data.prod.covid_sympthoms where contact_yes = 1 and age_60 = 1; </p>
    """

@app.route('/aggregator',methods=['GET','POST'])
def stats_collector():
    query_to_be_executed=request.args.get("query")
    extracted_data=run_query1(conn, query_to_be_executed)
    print(extracted_data)
    return extracted_data

@app.route('/data_fetcher',methods=['GET','POST'])
def data_extractor():
    query_to_be_executed = request.args.get("query")
    data_from_table = pd.read_sql(query_to_be_executed, conn)
    return {"data": json.loads(data_from_table.to_json(orient='records'))}


if __name__=='__main__':
    run_query(conn, statement_1)
    run_query(conn, statement4)
    app.run()